In [1]:
import tensorflow as tf
import numpy as np
import os
import ast
import abc
import random
from typing import List, Dict, Union, Tuple
import numpy as np
from scipy.stats import truncnorm
import argparse
import timeit
from datetime import datetime
# from tensorflow_addons.layers.nalu import NALU
%load_ext tensorboard

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # disable cuda sepeed up
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # disable CPU wornings

2022-11-04 17:25:32.771014: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-04 17:25:32.771157: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("-o", "--output", dest="output", default="naly_syn_simple_arith")
parser.add_argument("-d", "--dist", dest="dist", default="normal", help="Prob.Dist")
parser.add_argument("-p", "--params",dest="params" , default="(-3,3)", type=ast.literal_eval)
parser.add_argument("-e", "--ext",dest="ext" , default="(10,15)", type=ast.literal_eval)

parser.add_argument("-n", "--nalu", dest="nalu", default="nalui1")
parser.add_argument("-se", "--seed", dest="seed", default=42, type=int)
parser.add_argument("-op", "--operation", dest="op", default="MUL")



args = parser.parse_args([])

def sample(dist, params, numDim = 2, numDP = 64000):
    data = np.zeros(shape=(numDP, numDim))
    if dist == "normal":
        intmean = (params[0] + params[1]) / 2
        intstd = (params[1] - params[0]) / 6
        print(
            "Generating Data: \nInt: \tdist \t %s\n\t\tdata >=\t %s\n\t\tmean(s)\t %s\n\t\tdata <\t %s\n\t\tstd \t %s" % (
                dist, params[0], intmean, params[1], intstd))
        mi, ma = (params[0] - intmean) / intstd, (params[1] - intmean) / intstd
        data = np.reshape(truncnorm.rvs(mi, ma, intmean, intstd, size=numDim * numDP), data.shape)

    elif dist == "uniform":
        print("Generating Data: \nInt: \tdist \t %s\n\t\tdata >=\t %s\n\t\tdata <\t %s\n\t\t" % (
        dist, params[0], params[1]))
        data = np.reshape(np.random.uniform(params[0], params[1], size=numDim * numDP), data.shape)
    elif dist == "exponential":
        data = np.random.exponential(params, size=(numDP, numDim))
    else:
        raise Exception("Unknown distribution")
    data = np.reshape(data, [-1])  # reshape to mix both distributions per instance!
    np.random.shuffle(data)
    data = np.reshape(data, (numDP, numDim))
    return data


def operation(op, a, b):
    if op.lower() == "mul":
        return a * b
    if op.lower() == "add":
        return a + b
    if op.lower() == "sub":
        return a - b
    if op.lower() == "div":
        return a / b

random.seed(args.seed)
tf.random.set_seed(args.seed)
np.random.seed(args.seed)

data = sample(args.dist, args.params)
lbls = operation(args.op, data[:,0], data[:,1])
lbls = np.reshape(lbls, newshape=(-1, 1))

int_data = sample(args.dist, args.params)
int_lbls = operation(args.op, int_data[:,0], int_data[:,1])
int_lbls = np.reshape(int_lbls, newshape=(-1, 1))

ext_data = sample(args.dist, args.ext)
ext_lbls = operation(args.op, ext_data[:,0], ext_data[:,1])
ext_lbls = np.reshape(ext_lbls, newshape=(-1, 1))

Generating Data: 
Int: 	dist 	 normal
		data >=	 -3
		mean(s)	 0.0
		data <	 3
		std 	 1.0
Generating Data: 
Int: 	dist 	 normal
		data >=	 -3
		mean(s)	 0.0
		data <	 3
		std 	 1.0
Generating Data: 
Int: 	dist 	 normal
		data >=	 10
		mean(s)	 12.5
		data <	 15
		std 	 0.8333333333333334


In [3]:
BATCH_SIZE = 64

data_dp = tf.data.Dataset.from_tensor_slices((data, lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)
int_data_dp = tf.data.Dataset.from_tensor_slices((int_data, int_lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)
ext_data_dp = tf.data.Dataset.from_tensor_slices((ext_data, ext_lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)

2022-11-04 17:25:40.062706: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-04 17:25:40.062745: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-04 17:25:40.062775: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (filip-HP-ProBook-440-G3): /proc/driver/nvidia/version does not exist
2022-11-04 17:25:40.063135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
import tensorflow as tf
from typeguard import typechecked
from typing import List
from tensorflow_addons.utils import types


class NALURegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, reg_coef=0.1):
        self.reg_coef = reg_coef

    def __call__(self, var: List[tf.Variable]) -> tf.Tensor:
        return self.reg_coef * tf.reduce_mean(
            tf.math.maximum(tf.math.minimum(-var, var) + 20, 0)
        )

    def get_config(self):
        return {"reg_coef": float(self.reg_coef)}



# @tf.keras.utils.register_keras_serializable(package="Addons")
class NALU(tf.keras.layers.Layer):
    """Simplified NALU class to demonstrate lr problem."""

    @typechecked
    def __init__(
        self,
        input_dim: int,
        output_dim: int,
        w_initializer: types.Initializer = tf.random_normal_initializer(
            mean=0.88, stddev=0.2, seed=None
        ),
        m_initializer: types.Initializer = tf.random_normal_initializer(
            mean=0.5, stddev=0.2, seed=None
        ),
        g_initializer: types.Initializer = tf.random_normal_initializer(
            mean=0.0, stddev=0.2, seed=None
        ),
        *args,
        **kwargs,
    ):
        super(NALU, self).__init__(*args, **kwargs)

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.w_initializer = w_initializer
        self.m_initializer = m_initializer
        self.g_initializer = g_initializer

        # if-else statement is evaluated at the graph construction time, so no need to use tf.cond
        self.operator_weight_shape = [self.input_dim, self.output_dim]
        self.gate_shape = (self.output_dim, )

        # action variables
        self.w_hat = self.add_weight(
            name = "w",
            shape = self.operator_weight_shape, 
            dtype = tf.float32,
            initializer = self.w_initializer,
            trainable=True,
        )

        self.m_hat = self.add_weight(
            name = "m",
            shape = self.operator_weight_shape,
            dtype = tf.float32,
            initializer = self.m_initializer,
            trainable=True,
        )

        # gating varaible
        self.g = self.add_weight(
            name = "g",
            shape = self.gate_shape, 
            dtype = tf.float32,
            initializer = self.g_initializer,
            trainable=False,
        )

    def call(self, input):
        eps = 1e-7
        w1 = tf.math.tanh(self.w_hat) * tf.math.sigmoid(self.m_hat)
        a1 = tf.matmul(input, w1)

        m1 = tf.math.exp(tf.minimum(tf.matmul(tf.math.log(tf.maximum(tf.math.abs(input), eps)),w1), 20))
        
        # sign
        w1s = tf.math.abs(tf.reshape(w1, [-1]))
        xs = tf.concat([input] * w1.shape[1], axis=1)
        xs = tf.reshape(xs, shape=[-1, w1.shape[0] * w1.shape[1]])
        sgn = tf.sign(xs) * w1s + (1 - w1s)
        sgn = tf.reshape(sgn, shape=[-1, w1.shape[1], w1.shape[0]])
        ms = tf.math.reduce_prod(sgn, axis=2)
        
        g1 = tf.math.sigmoid(self.g)
        return g1 * a1 + (1 - g1) * m1 * tf.clip_by_value(ms, -1, 1)

### Learning rate for Adam optimiser in iNALU code is 0.01, but then the layer does not converge as in the case of original code.

After epoch 10 the results should be: \
\
loss: 2.14522E-07 &emsp; regularization-loss: 1.64449E+00 \
g: [0.00024883] \
w1: [[0.999989 ]
 [0.9999891]] \
ext loss: 1.72817E-03	&emsp; regularization-loss: 1.64447E+00 \
int loss: 2.03045E-07	&emsp; regularization-loss: 1.64447E+00 \
\
\
But they are: \
loss: 0.8376047611236572 &emsp; regularization-loss: 2.6134002208709717 \
\
g: [0.9922033] \
w1: [[0.0013842 ]
 [0.00519968]] \
ext loss: 24623.927734375 &emsp; regularization-loss: 2.6142208576202393 \
int loss: 0.9530942440032959 &emsp; regularization-loss: 2.614959955215454 \

Nevertheless, it's enought to change learning_step to **0.1** and then results are much closer to the original results: \
\
loss: 8.058979801717214e-06 &emsp; regularization-loss: 1.7113746404647827 \
g: [0.00167777] \
w1: [[0.9999924 ]
 [0.99999225]] \
ext loss: 0.05179274082183838 &emsp; regularization-loss: 1.7092827558517456 \
int loss: 8.268002602562774e-06 &emsp; regularization-loss: 1.7073990106582642


In [18]:
# you can modify marameter to see the difference
learning_rate = 0.1

g_initializer = tf.random_normal_initializer(mean=0, stddev=0.1, seed=args.seed)
w_initializer = tf.random_normal_initializer(mean=1, stddev=0.1, seed=args.seed)
m_initializer = tf.random_normal_initializer(mean=-1, stddev=0.1, seed=args.seed)

def regularize(var):
    return 1 * tf.maximum(tf.minimum(-var, var) + 20, 0)


model = NALU(2, 1, 
g_initializer =g_initializer,
w_initializer =w_initializer,
m_initializer =m_initializer,
)
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    grads = [tf.clip_by_value(g, -0.1, 0.1) for g in grads]
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value, 0.05 * tf.add_n([tf.reduce_mean(regularize(v)) for v in model.trainable_weights])

@tf.function
def train_step_gate(x, y):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.g)
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, [model.g])
    grads = [tf.clip_by_value(g, -0.1, 0.1) for g in grads]
    optimizer.apply_gradients(zip(grads, [model.g]))
    return loss_value, 0.05 * tf.add_n([tf.reduce_mean(regularize(v)) for v in [model.g]])


In [21]:
losses = []
reinitctr = 0
REGULARIZATION = True
for epoch in range(11):
    print("Epoch:",epoch)
    for i, (x,y) in enumerate(data_dp):
        if i % 10 < 8:
            loss_value, reg_loss = train_step(x, y)
        else:
            loss_value, reg_loss = train_step_gate(x, y)

        losses.append(loss_value)

        if i%1000 == 0:
            print(f"loss: {loss_value}, regularization-loss: {reg_loss}")
            print(f"g: {tf.math.sigmoid(model.g)}")
            print(f"w1: {tf.math.tanh(model.w_hat) * tf.math.sigmoid(model.m_hat)}")
            # #print(epoch, i, f"result: {model(x)}")
            eloss_ex, eloss_in = train_step(ext_data, ext_lbls), train_step(int_data, int_lbls)
            print(f"ext loss: {eloss_ex[0].numpy()}, regularization-loss: {eloss_ex[1].numpy()}")
            print(f"int loss: {eloss_in[0].numpy()}, regularization-loss: {eloss_in[1].numpy()}")

Epoch: 0
loss: 4.419816377776442e-06, regularization-loss: 0.9291782379150391
g: [0.00124801]
w1: [[0.99999857]
 [0.9999985 ]]
ext loss: 0.027524234727025032, regularization-loss: 0.9278391003608704
int loss: 4.5316910473047756e-06, regularization-loss: 0.9266332983970642
